In [2]:
from statsmodels.stats.power import NormalIndPower
import math
effect_size = 0.15*0.27/math.sqrt(0.27*(1-0.27))
ztest = NormalIndPower()
num = ztest.solve_power(
    effect_size = effect_size, # effect_size=(|μA-μB|)/σ 这是数值，但是标准差只能根据经验去预估；
    nobs1 = None,              # 对照组
    alpha = 0.05,              # alpha=α
    power = 0.8,               # power=1-β
    ratio = 1,                 # ratio=处理组与对照组的比值
    alternative = 'two-sided'
)
print(effect_size,num, num*1.5/0.9)

0.0912245460839306 1886.3105104135236 3143.850850689206


### 计算比值数据的特殊计算逻辑

In [12]:
# 当μA和μB都是比值时，效应量effect_size的计算方式有点不同：
# 2 * (arcsin(sqrt(μA)) - arcsin(sqrt(μB)))
import statsmodels as sm 
import statsmodels.api as sma 

p0 = 0.27
delta = 0.15

effect_size_api = sma.stats.proportion_effectsize(p0, (1-delta)*p0)
analysis = sm.stats.power.TTestIndPower()
result = analysis.solve_power(effect_size=effect_size_api,
                            alpha=0.05,
                            power=0.8,
                            alternative='two-sided')

deff = 1.5 # 设计效应1.5倍
enlarge = 0.1 # 失访率10%

print(effect_size_api)
print(f'简单随机抽样样本量 {result}')
print(f'考虑设计效应和失访率的样本量 {deff*result/(1-enlarge)}')

0.09363048548901032
简单随机抽样样本量 1791.5752273267883
考虑设计效应和失访率的样本量 2985.958712211314


# Power and Sample Size
statsmodels has a number of methods for power calculation

see e.g.: https://machinelearningmastery.com/statistical-power-and-power-analysis-in-python/

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

p0 = np.zeros(100)
p0[:27] = 1
p1 = np.zeros(100)
p1[:23] = 1

def perm_fun(x,nA,nB,double_site_test=True):
    n = nA + nB
    idx_B = set(np.random.choice(n, nB , replace=False))
    idx_A = set(range(n)) - idx_B
    if double_site_test:
        return abs(x[idx_B].mean() - x[idx_A].mean())
    else:
        return x[idx_B].mean() - x[idx_A].mean()

for n in range(900, 1300 , 100):
    beta = []
    for power_cal_times in tqdm(range(1000)):
        l = []
        for i in range(n):
            l.append(np.random.choice(p0,replace=True))
            l.append(np.random.choice(p1,replace=True))
        l = pd.Series(l)
        perm_diffs = [perm_fun(l,n,n) for _ in range(1000)]
        beta.append(np.mean([diff > 0.27*0.15 for diff in perm_diffs]))
    power_i = np.mean([betai < 0.05 for betai in beta])
    if power_i > 0.8:
        print(f'当样本量为{n}时，置换检验的p值为显著性的概率POWER为{power_i}')
    else:
        print(f'当样本量为{n}时，POWER={power_i}')


 16%|█▌        | 159/1000 [02:28<13:04,  1.07it/s]


KeyboardInterrupt: 

### 经验bootstrap方法是正确的；而百分位法没有依据
https://zhuanlan.zhihu.com/p/41099219

In [ ]:
from sklearn.utils import resample
import pandas as pd
from tqdm import tqdm
L = pd.Series([30,37,36,43,42,48,43,46,41,42])
print(L.mean())
results = []
for nrepeat in tqdm(range(100000)):
    sample = resample(L,replace=True)
    results.append(sample.mean())
results = pd.Series(results)
confidence_interval = list(results.quantile([0.025, 0.975]))
# 百分位bootstrap。这个无依据
print(confidence_interval[0], confidence_interval[1])

# 经验bootsrap方法1。正法
# results_delta = results.copy()-L.mean()
# print(L.mean()-list(results_delta.quantile([0.025, 0.975]))[1], L.mean()-list(results_delta.quantile([0.025, 0.975]))[0])

# 等价的经验boostrap方法2
print(2*L.mean()-confidence_interval[1], 2*L.mean()-confidence_interval[0])

  1%|          | 1038/100000 [00:00<00:09, 10370.61it/s]

40.8


100%|██████████| 100000/100000 [00:09<00:00, 10732.54it/s]

37.6 43.7
37.89999999999999 43.99999999999999
